1. Recorte das tabelas pertinentes do banco de dados

In [ ]:
import sqlite3

# Caminho do banco de dados original
original_db_path = "../data/DiscursosSenadores_02_05_2025.sqlite"
# Caminho do novo banco de dados
new_db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco de dados original
orig_conn = sqlite3.connect(original_db_path)
orig_cursor = orig_conn.cursor()

# Conectar ao novo banco de dados (será criado se não existir)
new_conn = sqlite3.connect(new_db_path)
new_cursor = new_conn.cursor()

# Lista das tabelas a serem copiadas
tables_to_copy = ["Discursos", "Senadores"]


# Função para criar e copiar tabelas específicas
def copy_table(cursor, new_cursor, table_name):
    # Obter schema da tabela
    cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table_name}';")
    create_table_sql = cursor.fetchone()[0]

    # Criar a tabela no novo banco de dados
    new_cursor.execute(create_table_sql)

    # Copiar dados
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()

    # Identificar colunas da tabela
    cursor.execute(f"PRAGMA table_info({table_name});")
    columns = [col[1] for col in cursor.fetchall()]
    placeholders = ", ".join(["?"] * len(columns))
    new_cursor.executemany(
        f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})", rows
    )


# Processar cada tabela
for table in tables_to_copy:
    copy_table(orig_cursor, new_cursor, table)

# Commit e fechar conexões
new_conn.commit()
orig_conn.close()
new_conn.close()

print(f"Novo banco de dados criado em: {new_db_path}")


2. Vacuum dos arquivos sqlite

In [ ]:
import sqlite3

# Arquivos SQLite a serem otimizados
files_to_vacuum = [
    "./DiscursosSenadores_02_05_2025_analisado.sqlite",
    "../data/DiscursosSenadores_02_05_2025.sqlite"
]

# Executa VACUUM em cada arquivo
for db_file in files_to_vacuum:
    print(f"Executando VACUUM no arquivo: {db_file}")
    conn = sqlite3.connect(db_file)
    conn.execute("VACUUM;")  # Executa o comando VACUUM
    conn.close()
    print(f"VACUUM concluído para {db_file}")

print("Processo completo!")


3. Incorporando análise recente

In [5]:
!pip install pandas openpyxl
import pandas as pd
import sqlite3

# Caminhos dos arquivos
excel_file = "./dados_analisados_2025-05-03_2.xlsx"
sqlite_db = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conexão com o banco de dados SQLite
conn = sqlite3.connect(sqlite_db)

# Carregar todas as planilhas do Excel como dataframes
excel_sheets = pd.read_excel(excel_file, sheet_name=None)  # Retorna um dicionário {nome_da_planilha: dataframe}

# Nomes das tabelas que **não devem ser criadas**
tables_to_exclude = {"Discursos", "Senadores"}

# Processar cada planilha
for sheet_name, df in excel_sheets.items():
    if sheet_name in tables_to_exclude:
        print(f"A planilha '{sheet_name}' foi ignorada porque já existe no banco de dados.")
        continue  # Ignorar tabelas que não devem ser importadas

    # Verificar se as colunas de chave existem na planilha
    if "CodigoParlamentar" in df.columns:
        df.set_index("CodigoParlamentar", inplace=False)  # Garantir chave primária
    if "CodigoPronunciamento" in df.columns:
        df.set_index("CodigoPronunciamento", inplace=False)

    # Importar o dataframe como tabela no SQLite
    print(f"Importando a planilha '{sheet_name}' como tabela no banco de dados...")
    df.to_sql(sheet_name, conn, if_exists="replace", index=False)  # Substituir tabela se já existir

print("Todas as planilhas foram importadas para o banco de dados!")

# Fechar conexão
conn.close()


  Obtaining dependency information for openpyxl from https://files.pythonhosted.org/packages/c0/da/977ded879c29cbd04de313843e76868e6e13408a94ed6b987245dc7c8506/openpyxl-3.1.5-py2.py3-none-any.whl.metadata
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Obtaining dependency information for et-xmlfile from https://files.pythonhosted.org/packages/c1/8b/5fe2cc11fee489817272089c4203e679c63b570a5aaeb18d852ae3cbba6a/et_xmlfile-2.0.0-py3-none-any.whl.metadata
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
A planilha 'Discursos' foi ignorada porque já existe no banco de dados.
A planilha 'Senadores' foi ignorada porque já existe no banco de dados.
Importando a planilha 'PesquisaSemantica' como tabela no banco de dados...
Importand

4. Verificação

In [6]:
import sqlite3

# Caminho do banco de dados
db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Excluir a tabela "Senadores"
table_to_exclude = "Senadores"

# Obter todas as tabelas do banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

# Filtrar a tabela Senadores
tables = [table for table in tables if table != table_to_exclude]

# Iterar sobre as tabelas e contar os CodigoPronunciamento distintos
for table in tables:
    try:
        cursor.execute(f"SELECT COUNT(DISTINCT CodigoPronunciamento) FROM {table};")
        count = cursor.fetchone()[0]
        print(f"Tabela '{table}' contém {count} CodigoPronunciamento distintos.")
    except sqlite3.OperationalError:
        print(f"Tabela '{table}' não possui a coluna 'CodigoPronunciamento'.")

# Fechar conexão
conn.close()


Tabela 'Discursos' contém 69519 CodigoPronunciamento distintos.
Tabela 'PesquisaSemantica' contém 67349 CodigoPronunciamento distintos.
Tabela 'AnaliseAnual' contém 12621 CodigoPronunciamento distintos.
Tabela 'TopicosCorpusTodo' contém 12680 CodigoPronunciamento distintos.
Tabela 'AnaliseCorpusTodo' contém 69519 CodigoPronunciamento distintos.


5. Adequação da tabela para análise

In [7]:
import sqlite3

# Caminho do banco de dados
db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Obter todas as tabelas do banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [row[0] for row in cursor.fetchall()]

# Iterar sobre as tabelas para modificar
for table in tables:
    # Obter as colunas da tabela
    cursor.execute(f"PRAGMA table_info({table});")
    columns_info = cursor.fetchall()
    columns = [col[1] for col in columns_info]

    # Ignorar as tabelas que não possuem a coluna "CodigoPronunciamento"
    if "CodigoPronunciamento" not in columns:
        print(f"Tabela '{table}' ignorada (não possui a coluna 'CodigoPronunciamento').")
        continue

    # Remover a coluna "index", se existir
    if "index" in columns:
        columns.remove("index")

    # Reordenar "CodigoPronunciamento" para ser a primeira coluna
    columns.remove("CodigoPronunciamento")
    new_columns_order = ["CodigoPronunciamento"] + columns

    # Criar uma nova tabela com a nova ordem de colunas
    new_table = f"{table}_new"
    columns_definition = ', '.join([f"{col} TEXT" for col in new_columns_order])  # Define novas colunas como texto
    cursor.execute(f"CREATE TABLE {new_table} ({columns_definition});")

    # Inserir dados da tabela antiga na nova ordem de colunas
    columns_select = ', '.join(new_columns_order)
    cursor.execute(f"INSERT INTO {new_table} ({columns_select}) SELECT {columns_select} FROM {table};")

    # Remover a tabela antiga e renomear a nova tabela
    cursor.execute(f"DROP TABLE {table};")
    cursor.execute(f"ALTER TABLE {new_table} RENAME TO {table};")

    print(
        f"Tabela '{table}' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).")

# Commit e fechar conexão
conn.commit()
conn.close()


Tabela 'Discursos' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).
Tabela 'Senadores' ignorada (não possui a coluna 'CodigoPronunciamento').
Tabela 'PesquisaSemantica' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).
Tabela 'AnaliseAnual' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).
Tabela 'TopicosCorpusTodo' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).
Tabela 'AnaliseCorpusTodo' modificada com sucesso (coluna 'CodigoPronunciamento' à esquerda e 'index' removida, se existente).


6. Adaptando tipos de dados das tabelas

In [10]:
import sqlite3

# Caminho do banco de dados
db_path = "./DiscursosSenadores_02_05_2025_analisado.sqlite"

# Conectar ao banco de dados SQLite
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Definições de tipos para colunas específicas ou que contenham palavras-chave
specific_columns_types = {
    "CodigoPronunciamento": "INTEGER",
    "CodigoParlamentar": "INTEGER",
    "DataPronunciamento": "DATE",
    "Ano": "DATE",
    "MencionaConstituicao": "BOOLEAN",  # Ou "INTEGER" se preferir
    "DownloadConcluido": "BOOLEAN"  # Ou "INTEGER" se preferir
}

# Palavras-chave para identificar colunas relevantes e o tipo associado
keywords_types = {
    "BM25": "FLOAT",
    "TFIDF": "FLOAT",
    "probabilidade": "FLOAT",
    "Probability": "FLOAT",
    "similaridade": "FLOAT"
}

# Nomes das tabelas específicas mencionadas
specific_tables = [
    "Discursos", "Senadores", "AnaliseAnual", "AnaliseCorpusTodo",
    "PesquisaSemantica", "TopicosCorpusTodo"
]


# Função para recriar uma tabela com modificações de colunas
def modify_table(cursor, table_name, existing_columns):
    # Criar a definição das novas colunas
    new_columns = []
    for column in existing_columns:
        col_name = column[1]  # Nome da coluna
        col_type = column[2]  # Tipo atual da coluna

        # Verificar se a coluna possui tipo definido (exatamente pelo nome)
        if col_name in specific_columns_types:
            new_type = specific_columns_types[col_name]
        else:
            # Verificar se a coluna contém palavras-chave nos nomes
            new_type = None
            for keyword, dtype in keywords_types.items():
                if keyword.lower() in col_name.lower():  # Comparação case insensitive
                    new_type = dtype
                    break

        # Se tipo for identificado, usar o novo; caso contrário, manter o tipo atual
        if new_type:
            new_columns.append(f"{col_name} {new_type}")
        else:
            new_columns.append(f"{col_name} {col_type}")

    # Criar tabela temporária com os novos tipos de colunas
    cursor.execute(f"CREATE TABLE {table_name}_temp ({', '.join(new_columns)});")

    # Copiar os dados para a nova tabela
    column_names = ", ".join([col[1] for col in existing_columns])
    cursor.execute(f"INSERT INTO {table_name}_temp SELECT {column_names} FROM {table_name};")

    # Substituir a tabela antiga pela nova
    cursor.execute(f"DROP TABLE {table_name};")
    cursor.execute(f"ALTER TABLE {table_name}_temp RENAME TO {table_name};")


# Localizar todas as tabelas do banco de dados
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
all_tables = [table[0] for table in cursor.fetchall()]

# Aplicar modificações para todas as tabelas ou tabelas específicas mencionadas
for table in all_tables:
    # Modificar tabelas específicas ou tabelas que começem com padrões como "Analise" ou "Topicos"
    if table in specific_tables or table.startswith("Analise") or table.startswith("Topicos"):
        cursor.execute(f"PRAGMA table_info({table});")
        existing_columns = cursor.fetchall()

        # Verificar se tabela possui colunas relevantes para modificação
        relevant_columns = any(
            col[1] in specific_columns_types or  # Nome exato
            any(keyword.lower() in col[1].lower() for keyword in keywords_types.keys())  # Nome contém palavra-chave
            for col in existing_columns
        )

        if relevant_columns:
            print(f"Modificando tabela: {table}")
            modify_table(cursor, table, existing_columns)

# Salvar alterações e fechar o banco
conn.commit()
conn.close()

print("Todas as alterações foram concluídas com sucesso!")


Modificando tabela: PesquisaSemantica
Modificando tabela: AnaliseAnual
Modificando tabela: TopicosCorpusTodo
Modificando tabela: AnaliseCorpusTodo
Modificando tabela: Discursos
Modificando tabela: Senadores
Todas as alterações foram concluídas com sucesso!
